# Xpand IT DS Challenge

This notebook contains the template you should use to present your code, results and conclusions. You should keep the main structure intact to make it easier to evaluate and compare in the end.

## Dataset
You can find the dataset in the data folder of the repository. The folder contains two files:
* dow_jones_index.data - dataset data
* dow_jones_index.names - dataset information and details

## Business Analysis
Here you should conduct a brief analysis of what is Dow Jones Index. You can enumerate the main topics to take into account based on the dataset provided as well as your understandings of the variables.


-----
The Dow Jones Index is a stock market index with 30 large companies traded in the USA. The dataset provided contains data from this index during the first two quarters of 2011. The time granularity is the week. We have 750 data points (rows), one for each pair (stock, week) during the 25 weeks compromising the two quarters.

Besides the quarter, stock and date indications, each row has information regarding three main dimensions of a stock: price, volume and dividends.

- Price:

    Each row contains the initial, final, maximum and minimum price of the stock in that week as well as the percentual variation between the initial and final price.
    
    There is also the initial and final price, and the percentual change, same as above, but for the next week.

- Volume:

    Each row carries info about the number of stocks (volume) traded during the current week, the previous one, as well as the percentual variation between both volumes.
    
- Dividends:

    Regarding dividends, for each row we know how many days for the next dividends payment as well as the percentual return of those dividends.

The exhaustive definition of the variable may be found [here](https://archive.ics.uci.edu/dataset/312/dow+jones+index).

-----


## Data Understanding
During the data understanding phase, you should focus on understanding what each variable represents, compute statistics and visualizations. Some questions that may guide your work follow:
* Feature engineering: should new features be created from the existing ones?
* What will be your features and your label?
* Is the dataset ready for the prediction task? (ex: missing values)
* How will the data be split into train and test sets?

-----

In [97]:
import numpy as np
import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [98]:
url = 'https://github.com/dsu-xpand-it/DSU-Recruitment-Challenges/blob/ea03a8a9cc386b65c4331ff0847091e7cb27d13b/data/dow_jones_index.data?raw=true'
df_raw = pd.read_csv(url)
df_raw

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2,XOM,5/27/2011,$80.22,$82.63,$80.07,$82.63,68230855,3.00424,-21.355713,86758820.0,$83.28,$81.18,-2.521610,75,0.568801
746,2,XOM,6/3/2011,$83.28,$83.75,$80.18,$81.18,78616295,-2.52161,15.221032,68230855.0,$80.93,$79.78,-1.420980,68,0.578960
747,2,XOM,6/10/2011,$80.93,$81.87,$79.72,$79.78,92380844,-1.42098,17.508519,78616295.0,$80.00,$79.02,-1.225000,61,0.589120
748,2,XOM,6/17/2011,$80.00,$80.82,$78.33,$79.02,100521400,-1.22500,8.811952,92380844.0,$78.65,$76.78,-2.377620,54,0.594786


After a quick look at the data, some aspects are easy to note:

- All rows from the first week lack the info about the previous week volume;

- For the same stock the 'volume' column in a week is the same as the 'previous_weeks_volume' for the following week, as expected;

- Some variables like the 'percent_change_price' are just a (deterministic) function of other variables, in this case the 'open' and 'close'. *Mutatis mutandis* for the 'percent_change_volume_over_last_wk' and the 'percent_change_next_weeks_price'.

In [99]:
(df_raw.query('date == "1/7/2011"').percent_change_volume_over_last_wk.unique(), df_raw.query('date == "1/7/2011"').previous_weeks_volume.unique())

(array([nan]), array([nan]))

For the sake of simplicity, we replace the 'date' column with a 'week' column with values from 1 to 25 representing the span of the weeks in the dataset, being the week 1 the first of the year.

The 6 columns regarding the prices of the stock (4 in the current week and 2 for the next week) need to be casted to numeric after removing the $ sign.

In [100]:
weeks = list(df_raw.date.unique())
w_dict = { weeks[i-1] : i for i in range(1, 26)}
df_silver = df_raw
df_silver['date'] = df_silver.date.apply(lambda x : w_dict[x])
df_silver = df_silver.rename(columns = {"date": "week"})

In [101]:
df_silver.open = pd.to_numeric(df_silver.open.apply(lambda x: x[1:]))
df_silver.high = pd.to_numeric(df_silver.high.apply(lambda x: x[1:]))
df_silver.low = pd.to_numeric(df_silver.low.apply(lambda x: x[1:]))
df_silver.close = pd.to_numeric(df_silver.close.apply(lambda x: x[1:]))
df_silver.next_weeks_open = pd.to_numeric(df_silver.next_weeks_open.apply(lambda x: x[1:]))
df_silver.next_weeks_close = pd.to_numeric(df_silver.next_weeks_close.apply(lambda x: x[1:]))

In [102]:
df_silver

,quarter,stock,week,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1,15.82,16.72,15.78,16.42,239655616,3.79267,NaN,NaN,16.71,15.97,-4.428490,26,0.182704
1,1,AA,2,16.71,16.71,15.64,15.97,242963398,-4.42849,1.380223,239655616.0,16.19,15.79,-2.470660,19,0.187852
2,1,AA,3,16.19,16.38,15.60,15.79,138428495,-2.47066,-43.024959,242963398.0,15.87,16.13,1.638310,12,0.189994
3,1,AA,4,15.87,16.63,15.82,16.13,151379173,1.63831,9.355500,138428495.0,16.18,17.14,5.933250,5,0.185989
4,1,AA,5,16.18,17.39,16.18,17.14,154387761,5.93325,1.987452,151379173.0,17.33,17.37,0.230814,97,0.175029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2,XOM,21,80.22,82.63,80.07,82.63,68230855,3.00424,-21.355713,86758820.0,83.28,81.18,-2.521610,75,0.568801
746,2,XOM,22,83.28,83.75,80.18,81.18,78616295,-2.52161,15.221032,68230855.0,80.93,79.78,-1.420980,68,0.578960
747,2,XOM,23,80.93,81.87,79.72,79.78,92380844,-1.42098,17.508519,78616295.0,80.00,79.02,-1.225000,61,0.589120
748,2,XOM,24,80.00,80.82,78.33,79.02,100521400,-1.22500,8.811952,92380844.0,78.65,76.78,-2.377620,54,0.594786


Here are some basic statistics about the numeric data, the number of missing values checks out for the columns based on the previous week volume.

In [103]:
df_silver.drop(columns=['quarter', 'week']).describe()

,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
count,750.000000,750.000000,750.000000,750.000000,7.500000e+02,750.000000,720.000000,7.200000e+02,750.000000,750.000000,750.000000,750.000000,750.000000
mean,53.651840,54.669987,52.640160,53.729267,1.175478e+08,0.050262,5.593627,1.173876e+08,53.702440,53.889080,0.238468,52.525333,0.691826
std,32.638852,33.215994,32.119277,32.788787,1.584381e+08,2.517809,40.543478,1.592322e+08,32.778111,33.016677,2.679538,46.335098,0.305482
min,10.590000,10.940000,10.400000,10.520000,9.718851e+06,-15.422900,-61.433175,9.718851e+06,10.520000,10.520000,-15.422900,0.000000,0.065574
25%,29.830000,30.627500,28.720000,30.365000,3.086624e+07,-1.288053,-19.804284,3.067832e+07,30.315000,30.462500,-1.222068,24.000000,0.534549
50%,45.970000,46.885000,44.800000,45.930000,5.306088e+07,0.000000,0.512586,5.294556e+07,46.015000,46.125000,0.101193,47.000000,0.681067
75%,72.715000,74.287500,71.037500,72.667500,1.327218e+08,1.650888,21.800622,1.333230e+08,72.715000,72.915000,1.845562,69.000000,0.854291
max,172.110000,173.540000,167.820000,170.580000,1.453439e+09,9.882230,327.408924,1.453439e+09,172.110000,174.540000,9.882230,336.000000,1.564210


Since the challenge is to find a way to pick a stock to invest in the following week, the columns about the next week prices are unknown in a real life scenario and cannot be used as features.

A simple approach would be to define the target as the column 'percent_change_next_weeks_price' and to train a time series regressor for that column that then allows one the simple pick the stock we highest predicted 'percent_change_next_weeks_price'. However, for that to be possible one would need more than the mere 25 time points that we have in this dataset.

Another possible approach, and the one we will use here, is to transform the stock picking problem in a binary classification problem. For each one of the 25 weeks, we will compute the median of the 'percent_change_next_weeks_price' between the 30 stocks, and label with a '1' the 15 that outperformed the median, and a '0' the 15 that underperformed the median. (We even get the bonus of having a label that is perfectly balanced in the dataset)

We will then train a classifier for the stock labels, and use it in a clever way (explained bellow) the pick the stock between the 30 possible choices.

In [104]:
# create label column
week_median = { w : df_silver.query(f'week == {w}')['percent_change_next_weeks_price'].median() for w in range(1,26)}
df_silver['label'] = df_silver.apply(lambda row : 1 if row.percent_change_next_weeks_price > week_median[row.week] else 0 , axis=1)
df_silver = df_silver.assign(label = lambda d: d['label'].astype('category'))

Given the particular problem at hands, the train test split will not be performed randomly between the 750 observations of the dataset, we will instead use the first 20 weeks as our training data, and the last 5 weeks as our test data. This will allow us to actually simulate the stock picking in the test weeks.

In [105]:
df_train = df_silver.query(' week <= 20 ')

Let's take a look at the correlation between the different variables:

In [106]:
corr = df_train.drop(columns=['quarter', 'stock', 'week']).corr()
corr.style.background_gradient(cmap='coolwarm').format(precision = 3)

,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend,label
open,1.000,1.000,0.999,0.999,-0.512,0.065,-0.008,-0.510,0.999,0.998,0.077,-0.100,-0.146,0.102
high,1.000,1.000,1.000,1.000,-0.511,0.083,-0.006,-0.510,1.000,0.999,0.078,-0.100,-0.149,0.101
low,0.999,1.000,1.000,1.000,-0.514,0.087,-0.018,-0.511,0.999,0.998,0.076,-0.101,-0.144,0.100
close,0.999,1.000,1.000,1.000,-0.514,0.102,-0.016,-0.510,1.000,0.999,0.077,-0.100,-0.147,0.101
volume,-0.512,-0.511,-0.514,-0.514,1.000,-0.151,0.196,0.876,-0.513,-0.513,-0.079,-0.061,-0.265,-0.098
percent_change_price,0.065,0.083,0.087,0.102,-0.151,1.000,-0.230,-0.085,0.102,0.102,-0.009,0.013,0.030,0.005
percent_change_volume_over_last_wk,-0.008,-0.006,-0.018,-0.016,0.196,-0.230,1.000,-0.133,-0.014,-0.016,-0.019,-0.040,-0.027,-0.031
previous_weeks_volume,-0.510,-0.510,-0.511,-0.510,0.876,-0.085,-0.133,1.000,-0.510,-0.511,-0.134,-0.050,-0.262,-0.095
next_weeks_open,0.999,1.000,0.999,1.000,-0.513,0.102,-0.014,-0.510,1.000,0.999,0.077,-0.101,-0.146,0.101
next_weeks_close,0.998,0.999,0.998,0.999,-0.513,0.102,-0.016,-0.511,0.999,1.000,0.113,-0.101,-0.145,0.125


The features used to train the model will be:

- percent_change_price
- percent_change_volume_over_last_wk
- days_to_next_dividend
- percent_return_next_dividend

In [107]:
df_gold = df_silver[['week', 'percent_change_price', 'percent_change_volume_over_last_wk', 'days_to_next_dividend', 'percent_return_next_dividend', 'label']]

df_train = df_gold.query(' week <= 20 ').drop(columns = ['week'])
df_test = df_gold.query(' week >= 21 ').drop(columns = ['week'])

x_train = df_train.drop(columns = ['label'])
x_test = df_test.drop(columns = ['label'])
y_train = df_train.label
y_test = df_test.label

## Modelling
In this phase, your main goal is to develop and describe your approach to the solution of the problem. Some guidelines to help you:
* What metrics will you use to evaluate your solutions?
* What are some algorithms that can lead to good results? And why?
* Describe in detail your thought process during the development of your solution.
* Present your results.


-----


Having a model to classify stocks as outperformers or underperformers in a particular week is not enough to pick the stock to invest in that week. The main ideia we will employ here is to use the probabilities given by the classifier to make the choice. Certainly there will be more than one stock classified as '1', among those we will pick the one with the highest probability (given by the model) of being '1'.

We will try a RandomForest classifier that as the advantage of not needing for the data to be scaled, since decision trees are not metric based algorithms.

In [108]:
# for the missing values we use the knn method for the imputation
imputer = KNNImputer(n_neighbors = 5)
imputer.fit(x_train)
x_train = imputer.transform(x_train)
x_test = imputer.transform(x_test) #irrelevante, nao ha nan no test, apenas por boa pratica

We will employ grid search to check several combination of hyperparameter for our model with 5-fold cross validation.

The reason for using the precision as the score of the tuning is related to the concrete problem in our hands: it is preferable for an *outperforming* stock to be labeled as an *underperforming* stock than vice versa, since we are trying to avoid picking underperforming stocks, even if we have no guaratee of picking the best stock.

In [109]:
param_grid = {
    'n_estimators' : [10, 20, 50],
    'max_depth': [10, 20, 50, None],
    'min_samples_leaf': range(1,4),
    'min_samples_split': range(2,6)
}

grid_search = GridSearchCV(RandomForestClassifier(random_state = 0), param_grid, cv=5, scoring = 'precision')

grid_search.fit(x_train, y_train)
grid_search.best_params_

{'max_depth': 50,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 20}

In [110]:
# model with best parameters
rf = RandomForestClassifier(random_state = 0, max_depth = 50, min_samples_leaf = 1, min_samples_split = 4, n_estimators = 20)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

In [111]:
print(
    accuracy_score(y_test, y_pred),
    f1_score(y_test, y_pred),
    precision_score(y_test, y_pred)
)

0.5466666666666666 0.569620253164557 0.5421686746987951


As we can see in by the metrics above, our model is far from ideal, nonetheless let's see how it behaves when picking stocks for last 5 weeks (test data).

First lets define a function to get the stock with higher probability of '1' by the model:

In [112]:
def pred_stock(week):
    df_week = df_silver.query(f'week == {week}')[['stock', 'percent_change_price', 'percent_change_volume_over_last_wk', 'days_to_next_dividend', 'percent_return_next_dividend']]
    stocks_prob = { row[0] : rf.predict_proba(row[1:].reshape(1, -1))[0,1] for row in df_week.values }
    return max(stocks_prob, key = lambda s : stocks_prob[s])

In week 21 our randomForest picked a 'good' stock, however there are 6 stocks better than this one:

In [113]:
# first week in test data:
print(pred_stock(21))
df_silver.query('week == 21')[['stock', 'percent_change_next_weeks_price', 'label']].sort_values('percent_change_next_weeks_price')

KRFT


,stock,percent_change_next_weeks_price,label
472,DIS,-6.014320,0
459,DD,-5.434370,0
420,CAT,-4.990130,0
407,BAC,-4.970510,0
368,AA,-4.841600,0
485,GE,-4.369920,0
706,UTX,-4.321840,0
498,HD,-4.284130,0
641,MSFT,-4.246790,0
381,AXP,-4.105860,0


Let's now compare the rate of return of picking stocks with our model with the average rate return among all 30 stocks during the 5 weeks of our test data. We will also compute the optimal rate of return (picking always the best stock) as well as the worst rate possible.

Since in each week with buy 100€ of the stock, the rate of return of the 5 weeks is the sum of the percent change of each week.

In [114]:
def get_return(week):
    stock = pred_stock(week)
    return df_silver.query(f"week == {week} and stock == '{stock}'").percent_change_next_weeks_price.values[0]

print('RF rate of return of last 5 weeks:', sum([get_return(week) for week in range(21, 26) ]),'%')

RF rate of return of last 5 weeks: 5.845169 %


In [115]:
stocks = list(df_silver.stock.unique())

avgs = [df_silver.query(f"week >= 21 and stock == '{stock}'").percent_change_next_weeks_price.sum()/30 for stock in stocks]

print('Average rate of return of last 5 weeks:', sum(avgs),'%')

Average rate of return of last 5 weeks: 0.18899298999999994 %


In [116]:
opt_ret = sum([df_silver.query(f"week == {week}").percent_change_next_weeks_price.max() for week in range(21,26)])
print('Best possible rate of return of last 5 weeks:', opt_ret,'%')

Best possible rate of return of last 5 weeks: 16.456369 %


In [117]:
wst_ret = sum([df_silver.query(f"week == {week}").percent_change_next_weeks_price.min() for week in range(21,26)])
print('Worst possible rate of return of last 5 weeks:', wst_ret,'%')

Worst possible rate of return of last 5 weeks: -16.85407 %


## Conclusions
In the conclusions, you should enumerate the results you got after completing the challenge.
* How good do you consider your results? 
* What are some factors that would contribute to get better results?
* What are some advantages and disadvantages of your solution?
* What can be done as future work to improve your results?


-----
The results are not optimal, however, the performance of the model in the desired task is well above average.

In order to improve the results, other aproaches can be employed. We have tried logistic regressions but with no success, however some neural network based models might be well suited for this task althoug the size of the dataset is probably a bit small for that approach.

Another viable option is to make use of genetic algorithms as in this [paper](https://www.researchgate.net/publication/236085839_Dynamic-Radius_Species-Conserving_Genetic_Algorithm_for_the_Financial_Forecasting_of_Dow_Jones_Index_Stocks).

-----

#### Feedback

-----
The challenge is interesting. It is a bit frustating before one realises that can solve this with binary classification since the classical regression algorithms to predict the next week price fail miserably with such a small dataset.

-----

To submit your solution you should e-mail us this notebook in response to the e-mail you initially received with the challenge.